# Loop test for parameter discovery

Before perfoming a voltage attack, the attacker must discover the parameters that increase their success probability.

First, we need to initialize the chipwhisper nano that will be responsible for reseting the board whenever it is necessary and the serial connection.

In [1]:
import chipwhisperer as cw
import time 

try:
    if not scope.connectStatus:
        scope.con()
except NameError:
    scope = cw.scope()

scope.default_setup()

# Initialize the FT232H
import serial
ser = serial.Serial('/dev/ttyUSB0',baudrate=115200)

# Function to reset the board
def reset_board():
    #Toggle reset
    ser.reset_input_buffer()
    scope.io.nrst = False
    time.sleep(0.05)
    scope.io.nrst = None #Pulls high by default
    time.sleep(0.05)

If during the test we remove and reconect the chipwhisper to the computer, we should run the following cell that will reconnect it.

In [16]:
scope.dis()
scope.con()
scope

ChipWhisperer Nano Device
fw_version = 
    major = 0
    minor = 65
    debug = 0
io = 
    tio1         = None
    tio2         = None
    tio3         = None
    tio4         = high_z
    pdid         = True
    pdic         = False
    nrst         = True
    clkout       = 0
    cdc_settings = bytearray(b'\x01\x00\x00\x00')
adc = 
    clk_src  = int
    clk_freq = 0
    samples  = 100000
glitch = 
    repeat     = 0
    ext_offset = 0

The following code will attempt different glitch widths and identify the ones that are able to glitch the device.

In [ ]:
from tqdm.notebook import trange

gc = cw.GlitchController(groups=["success", "reset", "normal"], parameters=["width"])
gc.display_stats()

g_step = 1
gc.set_global_step(g_step)
gc.set_range("width", 1, 10)
#gc.set_range("offset", 500,) # 4.15 microseconds 8,3ns * 500  

scope.vglitch_setup()
scope.glitch.repeat = 0
scope.glitch.ext_offset = 0

for glitch_setting in gc.glitch_values():
    scope.glitch.repeat = glitch_setting[0]
    scope.glitch.ext_offset = 500
    successes = 0
    resets = 0
    for i in range(3):
        reset_board()
        time.sleep(0.05)

        buf = ser.read_until(b'Starting for\r\n')
        print(buf)
        scope.arm()
        time.sleep(1)
        ser.write(b'c')

        # return True if capture timed out, false if it didn’t.
        if scope.capture():
            print("Capture failed!")
        else:
            time.sleep(0.05)
            while True:
                buf = ser.readline()
                #print(buf.decode())
                if b'Normal' in buf:
                    gc.add("normal")
                    #print("No glitch!:(")
                    break
                elif b'Glitch!' in buf:
                    print("Glitch!!")
                    print(buf)
                    print()
                    gc.add("success")
                    successes += 1
                    break
                elif b'Starting for' in buf:
                    #print("Reset")
                    gc.add("reset")
                    resets += 1
                    break                
    if successes > 0:                
        print("successes = {}, resets = {}, width = {}, ext_offset = {}".format(successes, resets, scope.glitch.repeat, scope.glitch.ext_offset))

print("Done glitching")